# lab10_NN, ReLU, Xavier, Dropout, and Adam using pytorch

Deep NN, ReLU, Xavier, Dropout, Adam 을 모두 구현한 pytorch

In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torch import nn, optim
import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn.init

In [23]:
#MNIST data
mnist_trainset = datasets.MNIST(root = './data',train =True,transform=transforms.ToTensor(),download=True)
mnist_testset = datasets.MNIST(root = './data', train = False, transform=transforms.ToTensor(),download=True)

In [24]:
# hyper parameters
training_epochs = 15
batch_size = 100
keep_drop = 0.7

# Data Loader (Input Pipeline)
train_loader = DataLoader(dataset=mnist_trainset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = DataLoader(dataset=mnist_testset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [27]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.dropout = torch.nn.Dropout2d(p = 1-keep_prob)
        
        self.l1 = torch.nn.Linear(784, 512,bias = True)
        torch.nn.init.xavier_uniform_(self.l1.weight)
        self.l2 = torch.nn.Linear(512, 512,bias = True)
        torch.nn.init.xavier_uniform_(self.l2.weight)
        self.l3 = torch.nn.Linear(512, 512,bias = True)
        torch.nn.init.xavier_uniform_(self.l3.weight)
        self.l4 = torch.nn.Linear(512, 512,bias = True)
        torch.nn.init.xavier_uniform_(self.l4.weight)
        self.l5 = torch.nn.Linear(512, 10,bias = True)
        torch.nn.init.xavier_uniform_(self.l5.weight)

    def forward(self, x):
        x = x.view(-1, 784)
        output = self.dropout(F.relu(self.l1(x)))
        output = self.dropout(F.relu(self.l2(output)))
        output = self.dropout(F.relu(self.l3(output)))
        output = self.dropout(F.relu(self.l4(output)))
        output = self.l5(output)
        F.softmax(output,dim = 1)
        return output


model = Model()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

NameError: name 'keep_prob' is not defined

In [26]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(len(mnist_trainset) / batch_size)
    model.train()

    for idx, (batch_xs, batch_ys) in enumerate(train_loader):
        optimizer.zero_grad()
        y_pred = model(batch_xs)
        loss = criterion(y_pred, batch_ys)
        loss.backward()
        optimizer.step()
        avg_cost += loss / total_batch
    print('\nTrain Epoch: {}\tLoss: {:.6f}'.format(epoch, avg_cost))
    

#         if idx % 10 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, idx * len(data), len(train_loader.dataset),
#                 100. * idx / len(train_loader), loss.data[0]))
    # TEST
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 0	Loss: 0.307409

Test set: Average loss: 0.0011, Accuracy: 9647/10000 (96%)

Train Epoch: 1	Loss: 0.143349

Test set: Average loss: 0.0010, Accuracy: 9704/10000 (97%)

Train Epoch: 2	Loss: 0.112172

Test set: Average loss: 0.0010, Accuracy: 9726/10000 (97%)

Train Epoch: 3	Loss: 0.093860

Test set: Average loss: 0.0008, Accuracy: 9759/10000 (97%)

Train Epoch: 4	Loss: 0.080106

Test set: Average loss: 0.0008, Accuracy: 9780/10000 (97%)

Train Epoch: 5	Loss: 0.072789

Test set: Average loss: 0.0007, Accuracy: 9804/10000 (98%)

Train Epoch: 6	Loss: 0.068293

Test set: Average loss: 0.0007, Accuracy: 9818/10000 (98%)

Train Epoch: 7	Loss: 0.061826

Test set: Average loss: 0.0007, Accuracy: 9805/10000 (98%)

Train Epoch: 8	Loss: 0.056961

Test set: Average loss: 0.0007, Accuracy: 9826/10000 (98%)

Train Epoch: 9	Loss: 0.056487

Test set: Average loss: 0.0008, Accuracy: 9799/10000 (97%)

Train Epoch: 10	Loss: 0.052338

Test set: Average loss: 0.0007, Accuracy: 9826/10000 (98%)